In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py


from holodeck import plot, detstats
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC
import holodeck as holo

import hasasia.sim as hsim

import sys
sys.path.append('/Users/emigardiner/GWs/holodeck/ecg-notebooks/parameter_investigation')
import anatomy as anat

<!-- # Get PSpace Info -->
a;sfdn

In [ ]:
# # use one file to get the shape
# npz_hard_time = np.load('/Users/emigardiner/GWs/holodeck/ecg-notebooks/parameter_investigation/anatomy_uniform09A_fullshape/hard_time_p0.5_0.5_0.5_0.5_0.5_0.5_s91_81_101.npz',
#                         allow_pickle=True)             
# print(npz_hard_time.files)
# SHAPE = npz_hard_time['shape']
# npz_hard_time.close()

# # get param names
# pspace = holo.param_spaces.PS_Uniform_09A(holo.log, nsamples=1, sam_shape=SHAPE, seed=None)
# param_names = pspace.param_names
# print(param_names)

# # set directory path
# sam_loc = '/Users/emigardiner/GWs/holodeck/ecg-notebooks/parameter_investigation/anatomy_uniform09A_fullshape/'
# save_dir=sam_loc+'/figures'       

### Vary parameters

In [ ]:
SHAPE=20
NREALS=30
NFREQS=40

def vary_parameter(
                   target_param,    # the name of the parameter, has to exist in `param_names`
                   params_list = [0.0, 0.5, 1.0],  # the values we'll check
                   pspace = holo.param_spaces.PS_Uniform_09A(holo.log, nsamples=1, sam_shape=SHAPE, seed=None),
                   pars=None, save_dir=None 
                   ):
    # get the parameter names from this library-space
    param_names = pspace.param_names
    num_pars = len(pspace.param_names)
    print(f"{num_pars=} :: {param_names=}")

    # choose each parameter to be half-way across the range provided by the library
    if pars is None:
        pars = 0.5 * np.ones(num_pars) 
    str_pars = str(pars).replace(" ", "_").replace("[", "").replace("]", "")
    # Choose parameter to vary
    param_idx = param_names.index(target_param)

    data = []
    params = []
    for ii, par in enumerate(params_list):
        pars[param_idx] = par
        print(f"{ii=}, {pars=}")
        # _params = pspace.param_samples[0]*pars
        _params = pspace.normalized_params(pars)
        params.append(_params)
        # construct `sam` and `hard` instances based on these parameters
        sam, hard = pspace.model_for_params(_params)
        if isinstance(hard, holo.hardening.Fixed_Time_2PL_SAM):
            hard_name = 'Fixed Time'
        elif isinstance(hard, holo.hardening.Hard_GW):
            hard_name = 'GW Only'
        # run this model, retrieving binary parameters and the GWB
        _data = holo.librarian.run_model(sam, hard, NREALS, NFREQS, 
                                        gwb_flag=False, singles_flag=True, params_flag=True, details_flag=True)
        data.append(_data)
    if save_dir is not None:
        str_shape = str(sam.shape).replace(", ", "_").replace("(", "").replace(")", "")
        filename = save_dir+'/%s_p%s_s%s.npz' % (target_param, str_pars, str_shape)
        np.savez(filename, data=data, params=params, hard_name=hard_name, shape=sam.shape, target_param=target_param )
        print('saved to %s' % filename)

    return (data, params, hard_name, sam.shape, target_param, filename)

In [ ]:
print(f"{data[1].keys()=}")
print(f"{params[1]=}")

# Timescales

In [ ]:
print(sings.par_names)
fobs_gw_cents = data['fobs_cents']
sspar = sings.all_sspars(fobs_gw_cents, data['sspar'])
bgpar = data['bgpar']
hard = holo.hardening.Fixed_Time_2PL_SAM()
print(data.keys())

In [ ]:
print(bgpar[0].shape)

In [ ]:
def tau_from_dadt(dadt, sepa):
    """ tau = dt/dlna = dt/(da/a) = a*(dt/da) = a/(da/dt)"""
    tau = (sepa)/dadt
    return tau
def tau_from_pars(sspar, bgpar, hard=holo.hardening.Hard_GW()):
    """ pars contain [mtot, mrat, redz_init, redz_final, dcom_final, sepa_final, angs_final]
    each for ss is in shape F,R,L and each for bg is in shape F,R"""
    dadt_ss = hard.dadt(sspar[0], sspar[1], sspar[5])
    dadt_bg = hard.dadt(bgpar[0], bgpar[1], bgpar[5])

    tau_ss = tau_from_dadt(dadt_ss, sspar[5])
    tau_bg = tau_from_dadt(dadt_bg, bgpar[5])
    return tau_ss, tau_bg 

tau_ss, tau_bg = tau_from_pars(sspar, bgpar, )



In [ ]:
data2 = rv_hard_time[0][1]
print(data2.keys())

# Plot Hardening Time vs. Separation

In [ ]:
print(sings.par_names)

In [ ]:
sepa_idx = np.where(sings.par_names=='sepa_final')[0]
print(sepa_idx)

In [ ]:
data = rv_hard_time[0][1]


xlabels = ['Binary Separation [pc]', 'GW Frequency [nHz]']
ylabels = ['Hardening Time [Gyr]', 'GW Characteristic Strain']

fig, axs = plot.figax(nrows=2, figsize=(5,6)
                     )
for ii,ax in enumerate(axs):
    ax.set_xlabel(xlabels[ii])
    ax.set_ylabel(ylabels[ii])

fig.tight_layout()




x1_bg = bgpar[sepa_idx]

